In [7]:
import json
import pandas as pd
import altair as alt
import numpy as np
import altair as alt
from vega_datasets import data
from altair import datum
def SystemB():
    df = pd.read_csv('Global_Superstore.csv', header=0)
    df['Price_per_unit'] = df.apply(lambda row: row['Sales'] / row['Quantity']/(1-row['Discount']), axis=1)
    df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d-%m-%Y') #transform 'Ship Date' row into datetime format
    df['Quarter'] = df['Ship Date'].dt.to_period('Q').astype(str)
    quarterly_counts = df.groupby(['Quarter','Sub-Category']).sum().reset_index()

    # data processing
    df_sample_SystemB = df.sample(frac=0.1, random_state=10) #sample 10% frothe original datasets
    alt.data_transformers.enable('data_server')

    df_subcat = df_sample_SystemB.groupby('Sub-Category')['Sales'].sum().reset_index()
    df_price = df_sample_SystemB[['Product Name', 'Sub-Category', 'Sales', 'Price_per_unit','Shipping Cost','Order Priority']]


    # generate quater data
    quarterly_data = quarterly_counts
    quarterly_data['start'] = quarterly_data.groupby('Sub-Category')['Sales'].cumsum() - quarterly_data['Sales']
    quarterly_data['end'] = quarterly_data.groupby('Sub-Category')['Sales'].cumsum()
    quarterly_data = quarterly_data[['Sub-Category', 'Quarter', 'start', 'end']]

    # the first chart
    selected_sub_cat = alt.selection_single(on='click', fields=['Sub-Category'], nearest=True)

    chart1 = alt.Chart(df_subcat).mark_bar().encode(
        x=alt.X('Sales:Q', axis=alt.Axis(title='Sales')),
        y=alt.Y('Sub-Category:N', axis=alt.Axis(title='Sub-Category')),
        color=alt.condition(selected_sub_cat, alt.value('purple'), alt.value('lightgray'))
    ).add_selection(selected_sub_cat).properties(width=400, height=255)



    # the second chart
    chart2 = alt.Chart(df_price).mark_circle().encode(
        x=alt.X('Product Name:N', axis=alt.Axis(title='Product Name', 
    #                                             labels=False,
                                                ticks=False,
    #                                             labelExpr="slice(datum.label, 0, 30)"
                                               )),
        y=alt.Y('Sales:Q', axis=alt.Axis(title='Sales')),
        size=alt.Size('Price_per_unit:Q', scale=alt.Scale(domain=(0, 300))),
        color=alt.Color('Price_per_unit:Q', scale=alt.Scale(scheme='purples')),
        tooltip=['Product Name:N', 'Price_per_unit:Q','Shipping Cost:Q','Order Priority:N'],
    ).transform_filter(
        selected_sub_cat
    ).transform_window(
        rank='rank()',
        sort=[alt.SortField('Product Name', order='descending')]
    ).transform_filter(
        alt.FieldLTEPredicate(field='rank', lte=100)
    ).properties(width=400, height=250)

    # the third chart
    chart3 = alt.Chart(quarterly_data).mark_bar(opacity=0.7).encode(
        x=alt.X('Quarter:O', axis=alt.Axis(title='Quarter')),
        y=alt.Y('start:Q', axis=alt.Axis(title='Quarter-on-quarter growth')),
        y2='end',
        color=alt.Color('Sub-Category:N', legend=alt.Legend(title='Sub-Category'), 
                        scale=alt.Scale(range=['#800080', '#8b008b', '#9932cc', '#9400d3', '#ba55d3', '#c71585', '#dda0dd', '#e6e6fa', '#ee82ee', '#ff00ff', '#ff1493', '#ff69b4', '#ffb6c1', '#ffc0cb', '#ffe4e1', '#fff0f5', '#fff5ee'], scheme='purples')),
    ).transform_filter(
        selected_sub_cat
    ).properties(width=850, height=200)

    # text = base.mark_text(align='center', baseline='bottom', dy=-5).encode(
    #     text=alt.Text('end-start:Q', format=".2f")
    # )

    # chart3 = alt.layer(base, text).properties(width=800, height=300)



    # concat two charts as obe system
    # chart = alt.vconcat(chart1, chart2, chart3).resolve_legend(
    #     color='independent'
    # ).properties(title='Sales and Unit Price by Sub-Category')
    # chart


    chart = alt.hconcat(chart1, chart2, spacing=20).resolve_legend(
        color='independent'
    )

    final_chart = alt.vconcat(chart, chart3).properties(title='Sales and Unit Price by Sub-Category')
    final_chart.save('SystemB.html')
    final_chart.show()
    #.show()



SystemB()

/var/folders/k2/fm2yt1695r140fn89kjr5yfm0000gn/T/ipykernel_67238/3592338465.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  quarterly_counts = df.groupby(['Quarter','Sub-Category']).sum().reset_index()


Displaying chart at http://localhost:55807/
